# PV System Distribution

In [1]:
# pyathena
from pyathena.connection import Connection
from pyathena.pandas_cursor import PandasCursor

In [2]:
cursor = Connection(
    profile_name="dev",
    region_name="us-west-2",
    s3_staging_dir="s3://nrel-tests/tracking_the_sun/"
).cursor(PandasCursor)

## 1. State Level

In [3]:
import math

# folium
from branca import colormap
import folium
from folium.plugins import MarkerCluster, FastMarkerCluster

# pandas & geopandas
import pandas
import geopandas

# shapely
from shapely import geometry

In [4]:
pv_state = cursor.execute(
    """
    SELECT state, COUNT(*) AS count
    FROM oedi.oedi_dev_tracking_the_sun
    GROUP by state;
    """
).as_pandas()

In [5]:
# convert pandas dataframe to geodataframe
geo_state = geopandas.read_file("us-states.geojson")[["id", "geometry"]]
geo_state.rename(columns={"id": "state"}, inplace=True)

geo_pv_state = geopandas.GeoDataFrame(
    data=pv_state.merge(geo_state, on="state"),
    geometry="geometry",
    crs={"init": "epsg:4326"}
)

In [6]:
# Display state pv system numbers on map
state_map = folium.Map(location=[39.8283, -98.5795], zoom_start=4, tiles="Mapbox Bright")

tooltip = folium.GeoJsonTooltip(
    fields=["count"],
    aliases=["count:"],
    labels=True,
    sticky=False
)

colorscale = colormap.LinearColormap(
    colors=["#ffffcc","#ffeda0","#fed976","#feb24c","#fd8d3c","#fc4e2a","#e31a1c","#bd0026","#800026"],
    index=[100, 500, 1000, 5000, 10000, 50000, 100000, 500000, 1000000]
)
def style_function(feature):
    count = feature["properties"]["count"]
    return {
        "color": "#000000",
        "weight": 0.2,
        "opacity": 0.6,
        "fillColor": colorscale(count),
        "fillOpacity": 0.4,
    }

folium.GeoJson(
    name="Distribution of State PV Systems",
    data=geo_pv_state.to_json(),
    tooltip=tooltip,
    style_function=style_function
).add_to(state_map)

state_map

## County Level

In [7]:
pv_county = cursor.execute(
    """
    SELECT state, county, COUNT(*) AS count
    FROM oedi.oedi_dev_tracking_the_sun
    GROUP by state, county;
    """
).as_pandas()

In [8]:
# convert pandas dataframe to geodataframe
geo_county = geopandas.read_file("us-counties.geojson")[["state", "county", "geometry"]]

# WARNING: pv_county contains invalid county name '-9999', use inner join here.
geo_pv_county = geopandas.GeoDataFrame(
    data=pandas.merge(pv_county, geo_county, how="inner", left_on=["state", "county"], right_on=["state", "county"]),
    geometry="geometry",
    crs={"init": "epsg:4326"}
)

In [9]:
# Display county pv system numbers on map
county_map = folium.Map(location=[39.8283, -98.5795], zoom_start=4, tiles="Mapbox Bright")

tooltip = folium.GeoJsonTooltip(
    fields=["count"],
    aliases=["count:"],
    labels=True,
    sticky=False
)

colorscale = colormap.LinearColormap(
    colors=["#ffffcc","#ffeda0","#fed976","#feb24c","#fd8d3c","#fc4e2a","#e31a1c","#bd0026","#800026"],
    index=[100, 500, 1000, 5000, 10000, 50000, 100000, 500000, 1000000]
)
def style_function(feature):
    count = feature["properties"]["count"]
    return {
        "color": "#000000",
        "weight": 0.2,
        "opacity": 0.6,
        "fillColor": colorscale(count),
        "fillOpacity": 0.4,
    }

folium.GeoJson(
    name="Distribution of State PV Systems",
    data=geo_pv_county.to_json(),
    tooltip=tooltip,
    style_function=style_function
).add_to(county_map)

county_map

## City Level

In [10]:
pv_city = cursor.execute(
    """
    SELECT state, county, city, COUNT(*) AS count
    FROM oedi.oedi_dev_tracking_the_sun
    GROUP by state, county, city;
    """
).as_pandas()

In [11]:
geo_city = geopandas.read_file("us-cities.geojson")[["state", "county", "city", "geometry"]]

In [12]:
# Handle case
pv_city["city"] = pv_city["city"].str.title()
pv_city["county"] = pv_city["county"].str.title()

geo_city["city"] = geo_city["city"].str.title()
geo_city["county"] = geo_city["county"].str.title()

# Merge
geo_pv_city = geopandas.GeoDataFrame(
    data=pandas.merge(pv_city, geo_city, how="inner", left_on=["state", "county", "city"], right_on=["state", "county", "city"]),
    geometry="geometry",
    crs={"init": "epsg:4326"}
)

In [13]:
# Display city pv system numbers on map
city_map = folium.Map(location=[39.8283, -98.5795], zoom_start=4, tiles="cartodbpositron")

marker_cluster = FastMarkerCluster(data=[[geom.y, geom.x] for geom in geo_pv_city.geometry.values])
marker_cluster.add_to(city_map)

city_map